In [2]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /gpfs/software/Anaconda/envs/ancIBD/lib/python3.11/site-packages/ancIBD-0.4-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import copy
import json
import math
from glob import glob
import scipy.spatial.distance as sciDist
from tqdm import tqdm
import requests
import time
import itertools
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import islice
from PIL import Image
import re
from tqdm import tqdm
from path_decomposition import linkMajor, computeSolSteps, linkMajor2B


# Headless simulator version
index = 0 # local server index 
API_ENDPOINT = 'http://localhost:4001/simulation-8bar' # NOT THE LS VERSION
HEADERS = {"Content-Type": "application/json"}
batchCount = 1 # Send this number of samples to MotionGen each time 
speedscale = 1
steps = 360
minsteps = int(steps*20/360)

mechType = 3

topo_numbers = []
with open('8bar.txt', 'r') as f:
    for line in f:
        if line.startswith('Topo '):
            parts = line.strip().split()
            if len(parts) >= 2 and parts[1].isdigit() and len(parts[1]) == 3:
                topo_numbers.append(parts[1])
print(topo_numbers)
# ['811', '812', '813', '814', '815', '816', '817', '818', '819', '821', '822', '823', '824', '825', '831', '832']

typesList = [f"Type{num}-" for num in topo_numbers]
print(typesList)
# ['Type811-', 'Type812-', 'Type813-', 'Type814-', 'Type815-', 'Type816-', 'Type817-', 'Type818-', 'Type819-', 'Type821-', 'Type822-', 'Type823-', 'Type824-', 'Type825-', 'Type831-', 'Type832-']

# shape of init pos 
types   = [typesList[mechType]]
initPos = [22] #11 pts, 22 coords
randSeed= [44] #ignore
couplerCurveIndices = [10]
distLens = [55] #ignore

output_dir = "outputs-8bar"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for t in typesList:
    dir_path = os.path.join(output_dir, t + "0")
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

['811', '812', '813', '814', '815', '816', '817', '818', '819', '821', '822', '823', '824', '825', '831', '832']
['Type811-', 'Type812-', 'Type813-', 'Type814-', 'Type815-', 'Type816-', 'Type817-', 'Type818-', 'Type819-', 'Type821-', 'Type822-', 'Type823-', 'Type824-', 'Type825-', 'Type831-', 'Type832-']


In [2]:
# good old ones 

def isValid(seq):
    if len(seq.shape) == 2:
        isVal = np.var(seq[:,0]) <= 5e-3 and np.var(seq[:,1]) <= 5e-3
    else:
        isVal = len(seq) == 0 or np.var(seq) <= 5e-3

    if isVal:
        return False
    else:
        return True


def get_pca_inclination(qx, qy, ax=None, label=''):
    """ Performs the PCA
        Return transformation matrix
    """
    cx = np.mean(qx)
    cy = np.mean(qy)
    covar_xx = np.sum((qx - cx)*(qx - cx))/len(qx)
    covar_xy = np.sum((qx - cx)*(qy - cy))/len(qx)
    covar_yx = np.sum((qy - cy)*(qx - cx))/len(qx)
    covar_yy = np.sum((qy - cy)*(qy - cy))/len(qx)
    covar = np.array([[covar_xx, covar_xy],[covar_yx, covar_yy]])
    eig_val, eig_vec= np.linalg.eig(covar)

    # Inclination of major principal axis w.r.t. x axis
    if eig_val[0] > eig_val[1]:
        phi= np.arctan2(eig_vec[1,0], eig_vec[0,0])
    else:
        phi= np.arctan2(eig_vec[1,1], eig_vec[0,1])

    return phi


def get_normalize_curve(jd, steps=None, rotations=1, normalize=True, transformParas=None):
    jd = np.array(jd)
    joint_data_n, x_mean, y_mean, denom, phi = [], None, None, None, None
    if isValid(jd):
        if steps:
            sample_indices = np.linspace(0, jd.shape[0]-1, steps, dtype=np.int32)
            jd = jd[sample_indices,:]
        if normalize:
            if not transformParas:
                x_mean = np.mean(jd[:,0], axis=0, keepdims=True)
                y_mean = np.mean(jd[:,1], axis=0, keepdims=True)
            else:
                x_mean, y_mean, denom, phi = transformParas
            jd[:,0] = jd[:,0] - x_mean
            jd[:,1] = jd[:,1] - y_mean

            if not transformParas:
                denom = np.sqrt(np.var(jd[:,0], axis=0, keepdims=True) + np.var(jd[:,1], axis=0, keepdims=True))
                denom = np.expand_dims(denom, axis=1)
            jd = jd / denom
            t = 0
        if not transformParas:
            phi = -get_pca_inclination(jd[:,0], jd[:,1])
        jd[:,0], jd[:, 1] = rotate_curve(jd, phi)
        for tt in range(rotations):
            joint_data_n.append(jd.copy())
            if rotations > 1:
                jd[:,0], jd[:,1] = rotate_curve(jd, t)
                t = 2*np.pi/rotations

    return joint_data_n, x_mean, y_mean, denom, phi


def rotate_curve(cur, theta):
    cpx = cur[:,0]*np.cos(theta) - cur[:,1]*np.sin(theta)
    cpy = cur[:,0]*np.sin(theta) + cur[:,1]*np.cos(theta)
    return cpx, cpy


def digitize_seq(nums, minlim, maxlim, bin_size=64):
    bins = np.linspace(minlim, maxlim, bin_size-1)
    nums_indices = np.digitize(nums, bins)
    return nums_indices


def get_normalize_joint_data_wrt_one_curve(joint_data, ref_ind = 4):
    ''' input s = [num_curves, num_points, 2]
    '''
    joint_data_n = []
    s = np.array(joint_data)

    if isValid(s[ref_ind]):
        x_mean = np.mean(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True)
        y_mean = np.mean(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True)
        s[:,:,0] = s[:,:,0] - x_mean
        s[:,:,1] = s[:,:,1] - y_mean
        denom = np.sqrt(np.var(s[ref_ind:ref_ind+1,:,0], axis=1, keepdims=True) + np.var(s[ref_ind:ref_ind+1,:,1], axis=1, keepdims=True))
        denom = np.expand_dims(denom, axis=2) #is this scale? 
        s = s / denom
        phi = -get_pca_inclination(s[ref_ind:ref_ind+1,:,0], s[ref_ind:ref_ind+1,:,1])
        for i in range(s.shape[0]):
            s[i,:,0], s[i,:,1] = rotate_curve(s[i], phi)
    else:
        return s, [None, None, None, None], False

    # s has a shape of (j_num, state, dim)
    return s, [x_mean[0][0], y_mean[0][0], denom[0][0][0], phi], True # tx, ty, scaling, rotation angle 


##############################################################################################
# There are some other necessary transformations. (x_mean, y_mean, phi, denom) are from get_normalize_curve. 
##############################################################################################
def get_image_from_point_cloud(points, xylim, im_size, inverted = True, label=None):
    mat = np.zeros((im_size, im_size, 1), dtype=np.uint8)
    x = digitize_seq(points[:,0], -xylim, xylim, im_size)
    if inverted:
        y = digitize_seq(points[:,1]*-1, -xylim, xylim, im_size)
        mat[y, x, 0] = 1
    else:
        y = digitize_seq(points[:,1], -xylim, xylim, im_size)
        mat[x, y, 0] = 1
    return mat


def process_mech_102723(jointData, ref_ind, im_size = 64, xylim = 3.5, inverted = True, swapAxes = True):
    paras = None

    # It is possible the jointData format is (angles, joint, (x, y)). 
    # You should put a True if this happens. (This is how files are saved).
    # I literally don't understand why I saved jointData with a shape of (angles, joint, (x, y)) 
    if swapAxes:
        jointData = np.swapaxes(jointData, 0, 1)

    # This converts all 
    jointData, paras, success = get_normalize_joint_data_wrt_one_curve(jointData, ref_ind= ref_ind)

    # jointData format from now on becomes np.array with a shape of (joint, curve_length, dimension)
    jointData = np.array(jointData)

    if success:
        # get binaryImage 
        jd = jointData[ref_ind]
        mat = get_image_from_point_cloud(jd, xylim=xylim, im_size=im_size, inverted=inverted)
        return mat, paras, success
    else: 
        return None, None, success

def calc_dist(coord):
    # Calculate differences using broadcasting
    diffs = coord[:, np.newaxis, :] - coord[np.newaxis, :, :]
    squared_dists = np.sum(diffs ** 2, axis=2)

    # Extract the upper triangle indices where i < j
    i, j = np.triu_indices(len(coord), k=1)
    dist_arr = np.sqrt(squared_dists[i, j])
    dist_arr = dist_arr/min(dist_arr)
    return np.round(dist_arr, 2)

In [3]:
def B2T(Bextend):
    n = len(Bextend[0])
    Textend = np.zeros((n,n))

    for i in range(n):
        if Bextend[0][i]:
            Textend[i][i] = 1

    for B in Bextend:
        for i in range(n):
            for j in range(i+1,n):
                if B[i] and B[j]:
                    Textend[i][j] = 1
                    Textend[j][i] = 1
    return Textend.astype(int).tolist()

In [4]:
from pprint import pprint
import copy 

def exchange_rows(i, j, cp, mat):
    mat_copy = copy.deepcopy(mat)
    mat_copy[cp][-1] = 1
    mat_copy[i], mat_copy[j] = mat_copy[j], mat_copy[i]
    return mat_copy

def exchange_columns(i, j, mat, pos):
    mat_copy = copy.deepcopy(mat)
    if pos[i] == j:
        return mat_copy, pos
    if pos[j] != j:
        j = pos.index(j)
    for row_i in range(len(mat_copy)):
        mat_copy[row_i][i], mat_copy[row_i][j] = mat_copy[row_i][j], mat_copy[row_i][i]
    
    pos[i], pos[j] = pos[j], pos[i]
    return mat_copy, pos

#--- replace below code with info from 8bar.txt corresponding to the 8 bar mech type
# Read 8bar.txt and extract the section for the current mechanism

# Use the current type string for the mechanism (e.g., 'Type811')
mech_number = types[index].replace('Type', '').replace('-', '')
mech_name = f"Topo {mech_number}"
section_found = False
section_lines = []

with open('8bar.txt', 'r') as f:
    for line in f:
        if line.strip().startswith('Topo') and mech_name in line:
            section_found = True
            continue
        if section_found:
            if line.strip().startswith('Topo') and mech_name not in line:
                break  # End of current section
            if line.strip() and not line.strip().startswith('Topo'):
                section_lines.append(line)

if not section_lines:
    raise ValueError(f"Section for {mech_name} not found or empty in 8bar.txt.")

# Prepare a local namespace to exec the lines
local_vars = {}
exec(''.join(section_lines), {}, local_vars)

# Assign the extracted variables, with error handling
try:
    B = local_vars['B']
    grounds = local_vars['grounds']
    actuator = local_vars['actuator']
    actuator_fixed = local_vars['actuator_fixed']
    chain = local_vars['chain']
    coupler_point = local_vars['coupler_point']
except KeyError as e:
    raise KeyError(f"Variable {e} not found in section for {mech_name} in 8bar.txt. Check the file format.")


'''B = [[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]


grounds = [0, 0, 1, 1, 5, 5, 6, 6, 2, 2, 2, 4, 4, 3, 3, 3]
actuator = [0, 2, 0, 1, 6, 7, 5, 4, 1, 3, 4, 6, 5, 2, 7, 3]
actuator_fixed = [2, 0, 1, 0, 7, 6, 4, 5, 3, 4, 1, 5, 6, 7, 3, 2]
chain = [1, [3, 7, 8], 2, [3, 4], [5, 9], [2, 3, 8], [6, 9], [1, 3], 0, [2, 7, 8], 5, 7, 4, 0, 6, [1, 4]]
coupler_point = [7, 6, 6, 7, 0, 1, 0, 7, 7, 7, 7, 0, 0, 6, 1, 6]'''

print(B)
print(grounds)
print(actuator)
print(actuator_fixed)
print(chain)
print(coupler_point)



[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]]
[0, 0, 5, 5, 7, 7, 2, 2, 3, 3, 3, 1, 1, 1, 4, 4, 4, 6, 6, 6]
[0, 2, 8, 5, 8, 9, 1, 7, 2, 4, 6, 0, 3, 1, 3, 4, 5, 9, 7, 6]
[2, 0, 5, 8, 9, 8, 7, 1, 4, 6, 4, 1, 0, 0, 4, 3, 4, 6, 6, 7]
[[1, 3], [4, 6], 9, [3, 4], 5, [6, 7], [0, 3], [6, 9], 0, [3, 5], [7, 9], 2, [4, 5], 7, [0, 1], [2, 6], 8, 8, 1, [2, 4]]
[7, 7, 2, 2, 2, 0, 5, 5, 5, 2, 5, 7, 7, 7, 7, 2, 2, 0, 5, 5]


In [5]:

#initStates = np.load("./npy-inputs/" + types[index] + '.npy')
initStates = np.load("/gpfs/scratch/raytang/walking-dataset-generator-old-simulator/Image-method-Synthesis-main/npy-inputs/RandPos-.npy")

couplerCurveIndex = couplerCurveIndices[index]
errCtr = 0
mechType = types[index]
batch = []
batchSaveStr = []
batchSaveNpyStr = []

def is_closed(pts):
    start_pt = pts[0]
    end_pt = pts[-1]
    path_dist = np.linalg.norm(end_pt - start_pt)
    is_closed = path_dist < 0.9
    return is_closed

for i, (g, a, af, c, cp) in enumerate(zip(grounds, actuator, actuator_fixed, chain, coupler_point)):
        
    B_new = exchange_rows(0, g, cp, B)
    col_positions = list(range(len(B[0])))
    
    B_new0, col_positions0 = exchange_columns(0, a, B_new, col_positions)
    B_new1, col_positions1 = exchange_columns(2, af, B_new0, col_positions0)
    B_new2, solSteps, c_final = None, None, None
    
    if type(c) == list:
        isOptim = False
        for ci in c:
            B_new2, rand = exchange_columns(1, ci, B_new1, col_positions1)

            _, solSteps, _ = computeSolSteps(linkMajor(B_new2))
            for solst in solSteps:
                if solst[1] == 'optim':
                    isOptim = True
                    break
            c_final = ci
            if not isOptim:
                break
                    
    else:
        B_new2, rand = exchange_columns(1, c, B_new1, col_positions1)
        _, solSteps, _ = computeSolSteps(linkMajor(B_new2))
        c_final = c


    T = B2T(B_new2)
    pprint(B_new2)
    print(mechType + str(i))
    pprint(solSteps)
    
    saveDir = os.path.abspath("./outputs-8bar/" + types[index] + str(i))
    saveDirNpy = os.path.abspath("./outputs-8bar/" + types[index] + str(i) + "-npy")

    distStore = [np.zeros(int(distLens[index]))]

    if not os.path.exists(saveDir):
        os.mkdir(saveDir)

    if not os.path.exists(saveDirNpy):
        os.mkdir(saveDirNpy)

    #for initState in tqdm(initStates):
    for initState in tqdm(initStates[:50000]):
        coord = np.round(initState, 3).reshape((int(initPos[index]/2),2))
        dist = calc_dist(coord)

        if max(dist) > 10:
            continue

        if np.any(np.all(dist == distStore, axis=1)):
            continue

        distStore.append(dist)
        param = coord.tolist()
        name = str(param).replace("[", "").replace("]", "").replace(",", "")

        exampleData = {
            'T': T, 
            'solSteps': solSteps, 
            'params': param,
            'speedScale':speedscale, # 1 
            'steps':steps, # 360 
            'relativeTolerance':0.1 
        }

        # The transformation 
        #np.save(saveDir + name + ' ' + types[index], param)

        batch.append(exampleData)
        batchSaveStr.append(saveDir + '/' + name + ' ' + types[index] + str(i) + ' ')
        batchSaveNpyStr.append(saveDirNpy + '/' + name + ' ' + types[index] + str(i) + ' ')

        if len(batch) >= batchCount:
            #print(batch[0], '\n', batch[1])
            #print(batchSaveStr[0], '\n', batchSaveStr[1])
            try:
                temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                time.sleep(0.02)
            except ValueError as v:
                for i in range(3):
                    time.sleep(2)
                    try:
                        temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                        break
                    except ValueError as v2:
                        errCtr += 1
            for i in range(len(temp)):
                P = np.array(temp[i]['poses'])
                np.save(batchSaveNpyStr[i] + '.npy', P)
                #reak
                try:
                    if len(P.shape) >= 1:
                        if P.shape[0] >= minsteps:
                            # do normalization, also get the transformation parameters. 
                            # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                            # This is just to avoid decimal difference problem 
                            imageMat, transParamSet, success = process_mech_102723(P, couplerCurveIndex)
                            if success and is_closed(P):
                                Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                                Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                                binary_data = np.uint8(imageMat[:,:,0]) * 255
                                img = Image.fromarray(binary_data)
                                img.save(batchSaveStr[i] + Tstr + '.jpg')
                                #plt.imshow(imageMat)
                                #plt.savefig(batchSaveStr[i] + Tstr + '.jpg')
                                #plt.clf()

                            pass                        

                except ValueError as v:
                    print(v)
                except FileNotFoundError as f:
                    print(f)
            batch = []
            batchSaveStr = []
            batchSaveNpyStr = []

    if len(batch) >= batchCount:
        #print(batch[0], '\n', batch[1])
        #print(batchSaveStr[0], '\n', batchSaveStr[1])
        try:
            temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
            time.sleep(0.02)
        except ValueError as v:
            for i in range(3):
                time.sleep(2)
                try:
                    temp = requests.post(url = API_ENDPOINT, headers=HEADERS, data = json.dumps(batch)).json()
                    break
                except ValueError as v2:
                        errCtr += 1
        for i in range(len(temp)):
            P = np.array(temp[i]['poses']) 
            np.save(batchSaveNpyStr[i] + '.npy', P)
            #print(P.shape)
            #reak
            try:
                if len(P.shape) >= 1:
                    if P.shape[0] >= minsteps:
                        # do normalization, also get the transformation parameters. 
                        # also the paras are saved instead of MP (M: tranformation matrix, P: points in the matrix)
                        # This is just to avoid decimal difference problem 
                        imageMat, transParamSet, success = process_mech_102723(P, couplerCurveIndex)
                        if success:
                            Tstr = np.array2string(np.round(transParamSet, 3), precision=3, suppress_small=True).replace("[", "").replace("]", "") # Wei asked for this part
                            Tstr = re.sub('\s+', ' ', Tstr) # to replace multiple sequential spaces together
                            binary_data = np.uint8(imageMat[:,:,0]) * 255
                            img = Image.fromarray(binary_data)
                            img.save(batchSaveStr[i] + Tstr + '.jpg')
                            #plt.imshow(imageMat)
                            #plt.savefig(batchSaveStr[i] + Tstr + '.jpg')
                            #plt.clf()
                            pass       
            except ValueError as v:
                print(v)
            except FileNotFoundError as f:
                print(f)
        batch = []
        batchSaveStr = []
        batchSaveNpyStr = []



[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
Type814-0
[[0, 'fixed', 0],
 [2, 'fixed', 2],
 [1, 'chain', 0],
 [3, 'arcSect', [0, 1]],
 [4, 'arcSect', [2, 3]],
 [6, 'arcSect', [2, 4]],
 [5, 'arcSect', [3, 4]],
 [7, 'arcSect', [1, 6]],
 [9, 'arcSect', [6, 7]],
 [8, 'arcSect', [5, 9]],
 [10, 'arcSect', [8, 9]]]


 42%|████▏     | 20843/50000 [01:39<02:19, 208.58it/s]


KeyboardInterrupt: 